In [ ]:
"""Урок 2. Операции над MANY-TO-MANY, CASCADE, LAZY"""


In [ ]:
"""
При добавлении детей родителю, либо родителей детям происходит запись в интеграционную таблицу. Добавляем элементы с помощью метода append - классика работы с листом, удаляем методом remove. 
Это супер удобно. То есть вроде бы создали третью таблицу, в которой храним связи, но при этом работа с ней происходит автоматически через таблицы, которые связаны с ней параметром secondary. Но, как обычно, есть нюансы. 
Допустим мы удаляем объект родителя, либо ребенка не через remove, а напрямую session.delete(son), как себя поведет интеграционная таблица?
Ответ - если есть связь через relationship() с таблицей родителей, то все будет хорошо и данные из интеграционной таблицы будут удалены. Но если relationship определен только в таблице родителей, то объект ребенка не будет знать, что ему нужно еще произвести удаление в интеграционной таблице, и, соответственно, удаление не произойдет. 


Наиболее эффективный вариант в таком случае - это настройка каскадного поведения. При нем операции по работе с объектами будут распространяться на таблицы, в которых есть ссылки на данные объекты.

Но прежде чем перейти к каскадам, хотел бы еще сказать про связь много-ко-многим. Наверняка, у большинства из вас возник вопрос: а почему нельзя было описать интеграционную таблицу в декларативном стиле? Ответ - можно, но есть ряд моментов, которые нужно учесть для корректного и удобного использования. Я предлагаю вам ознакомиться с данным решением самостоятельно, узнать как описывать объект для интеграционной таблицы и что из себя представляет association_proxy. 

Итак, атрибут cascade задается внутри конструкции relationship(), либо backref(), в случае, если мы описываем обратную ссылку.  Этот атрибут определяет поведение дочерних таблиц при выполнении операции над родителем. Давайте познакомимся с наиболее значимыми типами каскадного поведения, которые реализованы в SQLAlchemy.

save-update (по умолчанию) - при добавлении объекта в сессию (session.add(father)) объекты, связанные с ним через relationship() так же попадают в сессию.

То есть нам не нужно вызывать функцию add для каждого ребенка, которого мы создали и привязали к родителю, за нас это сделает сама ORM.

delete - при удалении родительского объекта, будут удалены все связанные с ним дети

По умолчанию, если каскад не имеет значения delete, то внешнее ключи у дочерних элементов обновляется на NULL.

delete-orphan - расширяет delete, при удалении ребенка из родительского объекта (father.children.remove(son)), произойдет удаление элемента из таблицы детей.

То есть удаляя сына у отца, объект сын будет удален из таблицы детей. Подразумевается, что у каждого дочернего объекта может быть только один родительский объект одновременно, и в подавляющем большинстве случаев он настроен только на отношение один-ко-многим.

Также есть типы merge, refresh-expire, expunge, которые используются достаточно редко, с ними вы можете познакомиться в документации пакета. 

Стоит отметить, что можно использовать значение all, которое объединяет в себя все типы, кроме delete-orphan. В атрибуте cascade типы задаются в строковом представлении, через запятую. Соответственно, если мы напишем all, delete-orphan, то мы сразу определим все каскадные типы для объекта. В таком случае мы укажем, что дочерний объект должен следовать вместе со своим родителем во всех случаях и удаляться, когда он больше не связан с этим родительским объектом.

Также, альтернативный вариант использования каскадного поведения объектов подразумевает описание поведения в объектах внешних ключей - ForeignKey, либо на уровне таблицы ForeignKeyConstraint - это сравнимо с описанием на уровне чистых SQL-конструкций. Настраиваемые атрибуты onupdate и ondelete позволяют определить как будут вести себя связанных объекты.

Еще одним интересным параметром для настройки поведение зависимых таблиц является параметр lazy. SQLAlchemy предоставляет возможности управления загрузкой связанных объектов при запросе. Под «связанными объектами» мы понимаем объекты, описанные с помощью конструкции relationship(). Это поведение можно настроить во время описания mapper с помощью атрибута lazy функции relationship(), а также с помощью параметров Query объекта.

Подгрузка зависимых таблиц делится на три категории: ленивая загрузка, жадная загрузка и отсутствие загрузки. 
ленивая загрузка (lazy, по умолчанию) - при запросе объекты загружаются без связанных таблиц. 
Когда обращается к конкретному связанному объекту, отправляется оператор SELECT на подгрузку;
жадная загрузка (нетерпеливая, eager) -  при запросе объекты загружаются вместе со связанными объектами;
Это достигается путем дополнения оператора SELECT, который он обычно генерирует с помощью JOIN для одновременной загрузки в связанных строках, либо путем испускания дополнительных операторов SELECT после основного
отсутствие загрузки (no loading) - отключению загрузки для связанных таблиц;
При обращении к связанной таблицы - атрибут пуст и никогда не подгружается, либо при обращении к нему вызывается ошибка, как защита от нежелательной ленивой загрузки.
Давайте поэкспериментируем с разными типами подгрузки на практике."""


from sqlalchemy import Column, Integer, ForeignKey, create_engine
from sqlalchemy.orm import relationship, sessionmaker, joinedload
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///lazy.db', echo=True)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


class Parent(Base):
   __tablename__ = 'parent'
   id = Column(Integer, primary_key=True)
   children = relationship("Child", lazy='select')


class Child(Base):
   __tablename__ = 'child'
   id = Column(Integer, primary_key=True)
   parent_id = Column(Integer, ForeignKey('parent.id'))


if __name__ == '__main__':
   Base.metadata.create_all(engine)

   parent = Parent()
   session.add(parent)
   child_one = Child(parent_id=1)
   child_two = Child(parent_id=1)
   session.add(child_one)
   session.add(child_two)
   session.commit()

   print('запрос родителя')
   my_parent = session.query(Parent).first()

   print('запрос детей')
   my_children = my_parent.children
   for c in my_children:
       print(c)

   print('custom lazy')
   q = session.query(Parent).options(joinedload(Parent.children)).all()


"""
При значение select у атрибута lazy подгрузка является ленивой загрузкой, как только мы захотим получить детей, так и выполнится дополнительный запрос

Значение joined является типом жадной загрузки, при запросе родителя применяется оператор JOIN для подгрузки детей.

Значение subquery - второй вид жадной загрузки. Эта форма загрузки Отправляется второй оператор SELECT, который дублирует исходный запрос. Этот запрос является подзапросом SQL-конструкции, которая подтягивает связанные объекты

selectin - еще один вид жадной загрузки. Создается еще один SELECT, который собирает идентификаторы родителя в конструкцию IN и внешний ключ parent_id ищет соответствия в этой конструкции

Два значения raise и noload являются типами отсутствия загрузки. В первом случае при обращении к связанным объектам вылетит ошибка, во втором None либо пустой список, в зависимости от типа связи.

Другой, не менее распространенный способ - настраивать подгрузку для каждого запроса на основе определенных атрибутов с помощью метода объекта Query - options(). Список атрибутов вы можете посмотреть в документации. 
"""